In [1]:
import numpy as np
from numpy import linalg as LA

import cvxpy as cp
import matplotlib.pyplot as plt

## Raw data

In [8]:
# http://files.grouplens.org/datasets/movielens/ml-100k-README.txt
!ls /private/home/qiantong/movielens100k/ml-100k

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item


In [5]:
# info
!cat /private/home/qiantong/movielens100k/ml-100k/u.info

943 users
1682 items
100000 ratings


In [7]:
# data : user id | item id | rating | timestamp
!head /private/home/qiantong/movielens100k/ml-100k/u.data

!wc -l /private/home/qiantong/movielens100k/ml-100k/u.data

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013
100000 /private/home/qiantong/movielens100k/ml-100k/u.data


In [11]:
# movie
!head /private/home/qiantong/movielens100k/ml-100k/u.item
!wc -l /private/home/qiantong/movielens100k/ml-100k/u.item

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

In [12]:
# user : user id | age | gender | occupation | zip code
!head /private/home/qiantong/movielens100k/ml-100k/u.user
!wc -l /private/home/qiantong/movielens100k/ml-100k/u.user

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703
943 /private/home/qiantong/movielens100k/ml-100k/u.user


In [14]:
!wc -l /private/home/qiantong/movielens100k/ml-100k/ua.base
!wc -l /private/home/qiantong/movielens100k/ml-100k/ua.test
!wc -l /private/home/qiantong/movielens100k/ml-100k/ub.base
!wc -l /private/home/qiantong/movielens100k/ml-100k/ub.test

90570 /private/home/qiantong/movielens100k/ml-100k/ua.base
9430 /private/home/qiantong/movielens100k/ml-100k/ua.test
90570 /private/home/qiantong/movielens100k/ml-100k/ub.base
9430 /private/home/qiantong/movielens100k/ml-100k/ub.test


## Load data

In [184]:
M = 943
N = 1682

Y = np.zeros([M, N])
Omega = {}

with open('/private/home/qiantong/movielens100k/ml-100k/u.data') as f:
    for line in f:
        sp = line.split('\t')
        user_id = int(sp[0]) - 1
        movie_id = int(sp[1]) - 1
        score = float(sp[2])
        
        Y[user_id][movie_id] = score
        Omega[(user_id, movie_id)] = score

print(Y)

[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


## Matrix completion

In [185]:
# X, Y have shape [M + N, M + N]

def f(X, Y):
    print(X[Y > 0])
    print(Y[Y > 0])
    return LA.norm(X[Y > 0] - Y[Y > 0]) ** 2 / 2

def nabla_f(X, Y):
    output = np.zeros(X.shape)
    output[Y > 0] = X[Y > 0] - Y[Y > 0]
    return output

def avg_error(X, Y):
    n_entries = np.sum(Y[Y > 0])
    return np.sum(np.abs(X[Y > 0] - Y[Y > 0])) / n_entries

def recover_error(X, Y, scale = 1):
    return (LA.norm(X * scale - Y) / LA.norm(Y)) ** 2

Y_noise = np.zeros([M, N])
Y_noise[Y > 0] = (Y + np.random.normal(0, 0.5, [M, N]))[Y > 0]
Y_large = np.zeros([M + N, M + N])
Y_large[:M, M:] = Y_noise
Y_large[M:, :M] = Y_noise.transpose()
print(Y_noise[Y > 0])
print(Y[Y > 0])

[5.73476032 3.26549051 3.78649988 ... 3.49116669 3.65653536 2.504497  ]
[5. 3. 4. ... 3. 3. 3.]


### cvxpy

In [149]:
print(cp.installed_solvers())

['ECOS', 'ECOS_BB', 'OSQP', 'SCS']


In [ ]:
TAU = 1000

W1 = cp.Variable((M, M), PSD=True)
W2 = cp.Variable((N, N), PSD=True)
X = cp.Variable((M, N))
X_large = cp.bmat([[W1, X], [X.T, W2]])

objective = cp.Minimize(cp.sum_squares(X_large[Y_large > 0] - Y_large[Y_large > 0]))
constraints = [cp.trace(W1) + cp.trace(W2) == TAU]
prob = cp.Problem(objective, constraints)

# prob.solve(solver=cp.MOSEK)
# f_hat = prob.solve()
f_hat = prob.solve(verbose=True, solver=cp.SCS, max_iters=10, eps=1e-2)
print("Solve time", prob.solver_stats.solve_time)
print("Optimal value", f_hat)

X_hat = X.value
f_hat = f(X_hat, Y)
print("f", f_hat)
err = recover_error(X_hat, Y)
print("Recover error", err)

In [182]:
print(X.value[Y > 0])
print(Y_noise[Y > 0])
print(Y[Y > 0])

[4.72086799 3.88911036 3.84826634 ... 3.71704566 3.32524181 3.37962296]
[4.72086799 3.88911036 3.84826634 ... 3.71704566 3.32524181 3.37962296]
[5. 3. 4. ... 3. 3. 3.]


### Our Algorithm

In [187]:
RANK = 1
TAU = 1

## Initialization

X = np.random.rand(M + N, RANK) - 0.5
X = X / LA.norm(X)
print("X", X.shape, LA.norm(X))
X = np.matmul(X, X.transpose())
X *= TAU
print("tr(X)", np.trace(X))    

## Run
eta = 0.1
epoch = 100
err_list = [recover_error(X, Y_large)]
pred_list = [f(X, Y_large)]

for i in range(epoch):
    # Compute V
    grad = nabla_f(X, Y_large)
    eig, vec = LA.eig(grad)
    idx = eig.argsort()[:RANK]
    vec = vec[:,idx]
    
    # Compute S
    S = cp.Variable((RANK, RANK), PSD=True)
    VSVT = vec * S * vec.transpose()
    new_X = (1 - eta) * X + eta * VSVT
    objective = cp.Minimize(cp.sum_squares(new_X[Y_large > 0] - Y_large[Y_large > 0]))
    constraints = [cp.trace(S) == TAU]
    prob = cp.Problem(objective, constraints)
    prob.solve(max_iters=10, eps=1e-2)

    S_hat = S.value
    VSVT = np.matmul(vec, S_hat)
    VSVT = np.matmul(VSVT, vec.transpose())
    
    # Update
    X = (1 - eta) * X + eta * VSVT
    
    err = recover_error(X, Y_large)
    pred = f(X, Y_large)
    err_list.append(err)
    pred_list.append(pred)
    print(i, eta, err, pred)

X (2625, 1) 1.0
tr(X) 1.0000000000000002
[-1.04662655e-04 -5.77426203e-05 -9.96433409e-05 ... -1.12078888e-04
 -1.91859025e-04  2.42081054e-05]
[5.73476032 3.26549051 3.78649988 ... 2.38997414 3.20342947 2.94772385]


KeyboardInterrupt: 